Khởi tạo SparkSession

In [2]:
from pyspark.sql import SparkSession
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
jar_path = "file:///C:/sedona-jars/sedona-spark-shaded-3.0_2.12-1.4.0.jar"

spark = SparkSession.builder \
    .appName("SedonaApp") \
    .config("spark.jars.packages", 
            "org.apache.sedona:sedona-python-adapter-3.0_2.12:1.4.1,"
            "org.apache.sedona:sedona-core-3.0_2.12:1.4.1,"
            "org.apache.sedona:sedona-sql-3.0_2.12:1.4.1") \
    .config("spark.jars.packages", "org.apache.sedona:sedona-python-adapter-3.0_2.12:1.4.1,org.datasyslab:geotools-wrapper:1.1.0-25.2") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryo.registrator", "org.apache.sedona.core.serde.SedonaKryoRegistrator") \
    .config("spark.sql.extensions", "org.apache.sedona.sql.SedonaSparkSessionExtensions") \
    .config("spark.serializer", KryoSerializer.getName) \
    .config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
    .getOrCreate()


In [4]:
from pyspark.sql.functions import expr

# 2. Truy cập vào JVM từ SparkSession
jvm = spark._jvm

# 3. Lấy đối tượng SedonaContext từ JVM
sedona_context = jvm.org.apache.sedona.spark.SedonaContext

# 4. Kích hoạt Sedona
sedona_context.create(spark._jsparkSession)

# 5. Đọc file CSV (Crime Data)
crime_df = spark.read.csv(
    "C:/Users/ADMIN/DATA ANALYST/Projectdata/Chicago_Crimes_100k.csv",
    header=True,
    inferSchema=True
)
#Đổi tên cột, bỏ khoảng trắng
for col in crime_df.columns:
    new_col = col.replace(' ', '')
    crime_df = crime_df.withColumnRenamed(col, new_col)

# 6. Tạo cột geom bằng ST_Point
crime_df = crime_df.withColumn(
    "geom",
    expr("ST_Point(cast(X as decimal(24,20)), cast(Y as decimal(24,20)))")
)
# 7. Kiểm tra schema để xác nhận cột geom đã được thêm vào
crime_df.printSchema()

# 8. Hiển thị một vài dòng đầu tiên để kiểm tra dữ liệu
crime_df.show(5)


root
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- ID: integer (nullable = true)
 |-- CaseNumber: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- PrimaryType: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- LocationDescription: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- CommunityArea: string (nullable = true)
 |-- FBICode: string (nullable = true)
 |-- XCoordinate: integer (nullable = true)
 |-- YCoordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- UpdatedOn: string (nullable = true)
 |-- geom: geometry (nullable = true)

+-------------+------------+-------+----------+--------------------+--------------------+----+-------------------+---------------

NHIỆM VỤ 2


In [23]:
# Đọc Parquet trong PySpark
crime_df = spark.read.parquet("C:/Users/ADMIN/DATA ANALYST/Projectdata/Chicago_Crimes_ZIP.parquet")


In [25]:
crime_df.createOrReplaceTempView("crimes")

# Truy vấn đúng: dùng ZIPCode thay vì zip_code
zip_counts_df = spark.sql("""
    SELECT ZIPCode as zip_code, COUNT(*) as crime_count
    FROM crimes
    GROUP BY ZIPCode
""")


In [13]:
# ----------------------------
# Bước 0: Import thư viện
# ----------------------------
import geopandas as gpd
import pandas as pd
from pyspark.sql import SparkSession

# ----------------------------
# Bước 1: Khởi tạo SparkSession
# ----------------------------
spark = SparkSession.builder \
    .appName("ZIPCode Crime Count") \
    .getOrCreate()

# ----------------------------
# Bước 2: Đọc dữ liệu Parquet và tạo bảng tạm
# ----------------------------
crime_df = spark.read.parquet("C:/Users/ADMIN/DATA ANALYST/Projectdata/Chicago_Crimes_ZIP.parquet")
crime_df.createOrReplaceTempView("crimes")

# ----------------------------
# Bước 3: Truy vấn đếm tội phạm theo ZIPCode
# ----------------------------
zip_counts_df = spark.sql("""
    SELECT ZIPCode, COUNT(*) as crime_coun
    FROM crimes
    WHERE ZIPCode IS NOT NULL
    GROUP BY ZIPCode
""")

# ----------------------------
# Bước 4: Chuyển sang Pandas để join với GeoDataFrame
# ----------------------------
crime_counts = zip_counts_df.toPandas()
crime_counts.rename(columns={'ZIPCode': 'ZIPCode'}, inplace=True)  # nếu cần đổi tên

# ----------------------------
# Bước 5: Đọc shapefile ZIP code
# ----------------------------
zip_shapes = gpd.read_file("C:/Users/ADMIN/DATA ANALYST/Projectdata/ZIPCodeCrimeCount/part-00000.shp")

# Kiểm tra tên cột ZIP
print("📌 Cột trong shapefile:", zip_shapes.columns)

# ----------------------------
# Bước 6: Join theo mã ZIP
# ----------------------------
# Thay 'ZIPCode' bằng tên thực tế trong shapefile nếu khác (ví dụ 'ZCTA5CE10')
merged = zip_shapes.merge(crime_counts, on='ZIPCode', how='inner')

# ----------------------------
# Bước 7: Xuất kết quả
# ----------------------------
# Ghi shapefile mới để dùng trong QGIS
output_shapefile = "C:/Users/ADMIN/DATA ANALYST/Projectdata/ZIPCodeCrimeCount/ZIP_CrimeCount.shp"
merged.to_file(output_shapefile)
print(f"✅ Đã ghi xong shapefile: {output_shapefile}")

# (Tuỳ chọn) Ghi thêm CSV nếu cần
output_csv = "C:/Users/ADMIN/DATA ANALYST/Projectdata/ZIPCodeCrimeCount/ZIP_CrimeCount.csv"
merged.drop(columns='geometry').to_csv(output_csv, index=False)
print(f"✅ Đã ghi xong CSV: {output_csv}")


📌 Cột trong shapefile: Index(['ZIPCode', 'count', 'geometry'], dtype='object')
✅ Đã ghi xong shapefile: C:/Users/ADMIN/DATA ANALYST/Projectdata/ZIPCodeCrimeCount/ZIP_CrimeCount.shp
✅ Đã ghi xong CSV: C:/Users/ADMIN/DATA ANALYST/Projectdata/ZIPCodeCrimeCount/ZIP_CrimeCount.csv
